In [82]:
import pandas as pd
import numpy as np
from IPython.display import display
from cmath import isnan
from numpy import dot
from numpy.linalg import norm
from collections import defaultdict
import plotly.express as px

#### Получить исходные данные

In [83]:
pd.set_option('display.max_columns', None)
df0 = pd.read_csv('data.csv', delimiter=';', encoding="utf8")
df = df0.copy(deep=True)

#### Преобразовать параметры (пол, размер)

In [84]:
genderDict = { "м": 0, "ж": 1 }
df["Пол"] = df["Пол"].map(lambda elem: genderDict[elem])

sizeDict = { "S": 42, "M": 44, "L": 46, "XL": 48, "XXL": 50 }
df["Размер"] = df["Размер"].map(lambda elem: sizeDict[elem])

df["Иерархия"] = df["Иерархия"].map(lambda elem: elem.split(","))

In [85]:
F_NAME = "Название"
F_DIST = "Расстояние"

nameArr = df["Название"]

dfTree = pd.DataFrame(
    columns=["Название", "Иерархия"], 
    data=df[["Название", "Иерархия"]].values
)

In [86]:
display(df)

del df["Название"]
del df["Иерархия"]

,Название,Пол,Цвет,Цена,Размер,Бренд,Иерархия
0,Брюки Briony Голубой 13300,1,Голубой,13300,42,Briony,"[Деловая одежда, Брюки]"
1,Брюки Corneliani Синий 14500,0,Синий,14500,50,Corneliani,"[Деловая одежда, Брюки]"
2,Брюки Hugo Boss Серый 14200,0,Серый,14200,46,Hugo Boss,"[Деловая одежда, Брюки]"
3,Брюки Canali Серый 11400,1,Серый,11400,44,Canali,"[Деловая одежда, Брюки]"
4,Брюки Briony Черный 13800,1,Черный,13800,44,Briony,"[Деловая одежда, Брюки]"
...,...,...,...,...,...,...,...
123,Шорты Lacoste Розовый 2700,1,Розовый,2700,50,Lacoste,"[Повседневная одежда, Шорты]"
124,Шорты H&M Красный 2600,0,Красный,2600,46,H&M,"[Повседневная одежда, Шорты]"
125,Шорты Zara Розовый 3600,1,Розовый,3600,46,Zara,"[Повседневная одежда, Шорты]"
126,Шорты Levi's Синий 2300,0,Синий,2300,42,Levi's,"[Повседневная одежда, Шорты]"


#### Задать матрицу смежности для брендов

In [87]:
businessBrands = {
    "Briony": 0,
    "Corneliani": 1,
    "Hugo Boss": 2,
    "Canali": 3,
}

everydayBrands = {
    "H&M": 4,
    "Zara": 5,
    "Levi's": 6,
    "Lacoste": 7,
}

sportsBrands = {
    "Nike": 8,
    "Adidas": 9,
    "Puma": 10,
    "Reebok": 11,
}

brands = {
    **businessBrands,
    **everydayBrands,
    **sportsBrands,
}

countBrands = len(businessBrands) + len(everydayBrands) + len(sportsBrands)

brandsMatr = np.zeros((countBrands, countBrands))

for businessBrandKey in businessBrands.keys():
    for _businessBrandKey in businessBrands.keys():
        if _businessBrandKey != businessBrandKey:
            brandsMatr[businessBrands[businessBrandKey], businessBrands[_businessBrandKey]] = \
            brandsMatr[businessBrands[_businessBrandKey], businessBrands[businessBrandKey]] = 0.1

    for everydayBrandKey in everydayBrands.keys():
        brandsMatr[businessBrands[businessBrandKey], everydayBrands[everydayBrandKey]] = \
        brandsMatr[everydayBrands[everydayBrandKey], businessBrands[businessBrandKey]] = 0.6

    for sportsBrandKey in sportsBrands.keys():
        brandsMatr[businessBrands[businessBrandKey], sportsBrands[sportsBrandKey]] = \
        brandsMatr[sportsBrands[sportsBrandKey], businessBrands[businessBrandKey]] = 1

for everydayBrandKey in everydayBrands.keys():
    for _everydayBrandKey in everydayBrands.keys():
        if _everydayBrandKey != everydayBrandKey:
            brandsMatr[everydayBrands[everydayBrandKey], everydayBrands[_everydayBrandKey]] = \
            brandsMatr[everydayBrands[_everydayBrandKey], everydayBrands[everydayBrandKey]] = 0.1
            
    for sportsBrandKey in sportsBrands.keys():
        brandsMatr[everydayBrands[everydayBrandKey], sportsBrands[sportsBrandKey]] = \
        brandsMatr[sportsBrands[sportsBrandKey], everydayBrands[everydayBrandKey]] = 0.4

for sportsBrandKey in sportsBrands.keys():
    for _sportsBrandKey in sportsBrands.keys():
        if _sportsBrandKey != sportsBrandKey:
            brandsMatr[sportsBrands[sportsBrandKey], sportsBrands[_sportsBrandKey]] = \
            brandsMatr[sportsBrands[_sportsBrandKey], sportsBrands[sportsBrandKey]] = 0.1

#### Задать матрицу смежности для цветов

In [88]:
colors = {
    "Белый": 0,
    "Черный": 1,
    "Серый": 2,
    "Синий": 3,
    "Голубой": 4,
    "Красный": 5,
    "Розовый": 6
}

colorsMatr = np.zeros((len(colors), len(colors)))

colorsMatr[colors["Белый"], colors["Черный"]] = colorsMatr[colors["Черный"], colors["Белый"]] = 1
colorsMatr[colors["Белый"], colors["Серый"]] = colorsMatr[colors["Серый"], colors["Белый"]] = 0.5
colorsMatr[colors["Белый"], colors["Синий"]] = colorsMatr[colors["Синий"], colors["Белый"]] = 0.7
colorsMatr[colors["Белый"], colors["Голубой"]] = colorsMatr[colors["Голубой"], colors["Белый"]] = 0.4
colorsMatr[colors["Белый"], colors["Красный"]] = colorsMatr[colors["Красный"], colors["Белый"]] = 0.7
colorsMatr[colors["Белый"], colors["Розовый"]] = colorsMatr[colors["Розовый"], colors["Белый"]] = 0.4

colorsMatr[colors["Черный"], colors["Серый"]] = colorsMatr[colors["Серый"], colors["Черный"]] = 0.3
colorsMatr[colors["Черный"], colors["Синий"]] = colorsMatr[colors["Синий"], colors["Черный"]] = 0.6
colorsMatr[colors["Черный"], colors["Голубой"]] = colorsMatr[colors["Голубой"], colors["Черный"]] = 1
colorsMatr[colors["Черный"], colors["Красный"]] = colorsMatr[colors["Красный"], colors["Черный"]] = 0.6
colorsMatr[colors["Черный"], colors["Розовый"]] = colorsMatr[colors["Розовый"], colors["Черный"]] = 1

colorsMatr[colors["Серый"], colors["Синий"]] = colorsMatr[colors["Синий"], colors["Серый"]] = 0.8
colorsMatr[colors["Серый"], colors["Голубой"]] = colorsMatr[colors["Голубой"], colors["Серый"]] = 1
colorsMatr[colors["Серый"], colors["Красный"]] = colorsMatr[colors["Красный"], colors["Серый"]] = 0.8
colorsMatr[colors["Серый"], colors["Розовый"]] = colorsMatr[colors["Розовый"], colors["Серый"]] = 1

colorsMatr[colors["Синий"], colors["Голубой"]] = colorsMatr[colors["Голубой"], colors["Синий"]] = 0.3
colorsMatr[colors["Синий"], colors["Красный"]] = colorsMatr[colors["Красный"], colors["Синий"]] = 0.7
colorsMatr[colors["Синий"], colors["Розовый"]] = colorsMatr[colors["Розовый"], colors["Синий"]] = 1

colorsMatr[colors["Голубой"], colors["Красный"]] = colorsMatr[colors["Красный"], colors["Голубой"]] = 1
colorsMatr[colors["Голубой"], colors["Розовый"]] = colorsMatr[colors["Розовый"], colors["Голубой"]] = 0.7

colorsMatr[colors["Красный"], colors["Розовый"]] = colorsMatr[colors["Розовый"], colors["Красный"]] = 0.3

#### Матрица смежности для категорий (1 уровень)

In [89]:
layer1 = {
    "Деловая одежда": 0,
    "Спортивная одежда": 1,
    "Повседневная одежда": 2,
}

treeLayer1 = np.zeros((len(layer1), len(layer1)))

treeLayer1[layer1["Деловая одежда"], layer1["Спортивная одежда"]] = treeLayer1[layer1["Спортивная одежда"], layer1["Деловая одежда"]] = 1
treeLayer1[layer1["Деловая одежда"], layer1["Повседневная одежда"]] = treeLayer1[layer1["Повседневная одежда"], layer1["Деловая одежда"]] = 0.6

treeLayer1[layer1["Спортивная одежда"], layer1["Повседневная одежда"]] = treeLayer1[layer1["Повседневная одежда"], layer1["Спортивная одежда"]] = 0.4


#### Матрица смежности для категорий (2 уровень)

In [90]:
layer2 = {
    "Пиджаки": 0,
    "Брюки": 1,
    "Рубашки": 2,
    "Спортивные футболки": 3,
    "Спортивные шорты": 4,
    "Спортивные брюки": 5,
    "Спортивные толстовки": 6,
    "Свитеры и кофты": 7,
    "Футболки": 8,
    "Лонгсливы": 9,
    "Джинсы": 10,
    "Шорты": 11,
}

treeLayer2 = np.zeros((len(layer2), len(layer2)))

treeLayer2[layer2["Пиджаки"], layer2["Брюки"]] = treeLayer2[layer2["Брюки"], layer2["Пиджаки"]] = 0.3
treeLayer2[layer2["Пиджаки"], layer2["Рубашки"]] = treeLayer2[layer2["Рубашки"], layer2["Пиджаки"]] = 0.2
treeLayer2[layer2["Пиджаки"], layer2["Спортивные футболки"]] = treeLayer2[layer2["Спортивные футболки"], layer2["Пиджаки"]] = 1
treeLayer2[layer2["Пиджаки"], layer2["Спортивные шорты"]] = treeLayer2[layer2["Спортивные шорты"], layer2["Пиджаки"]] = 1
treeLayer2[layer2["Пиджаки"], layer2["Спортивные брюки"]] = treeLayer2[layer2["Спортивные брюки"], layer2["Пиджаки"]] = 1
treeLayer2[layer2["Пиджаки"], layer2["Спортивные толстовки"]] = treeLayer2[layer2["Спортивные толстовки"], layer2["Пиджаки"]] = 1
treeLayer2[layer2["Пиджаки"], layer2["Свитеры и кофты"]] = treeLayer2[layer2["Свитеры и кофты"], layer2["Пиджаки"]] = 0.7
treeLayer2[layer2["Пиджаки"], layer2["Футболки"]] = treeLayer2[layer2["Футболки"], layer2["Пиджаки"]] = 0.6
treeLayer2[layer2["Пиджаки"], layer2["Лонгсливы"]] = treeLayer2[layer2["Лонгсливы"], layer2["Пиджаки"]] = 0.6
treeLayer2[layer2["Пиджаки"], layer2["Джинсы"]] = treeLayer2[layer2["Джинсы"], layer2["Пиджаки"]] = 0.5
treeLayer2[layer2["Пиджаки"], layer2["Шорты"]] = treeLayer2[layer2["Шорты"], layer2["Пиджаки"]] = 1

treeLayer2[layer2["Брюки"], layer2["Рубашки"]] = treeLayer2[layer2["Рубашки"], layer2["Брюки"]] = 0.3
treeLayer2[layer2["Брюки"], layer2["Спортивные футболки"]] = treeLayer2[layer2["Спортивные футболки"], layer2["Брюки"]] = 0.9
treeLayer2[layer2["Брюки"], layer2["Спортивные шорты"]] = treeLayer2[layer2["Спортивные шорты"], layer2["Брюки"]] = 1
treeLayer2[layer2["Брюки"], layer2["Спортивные брюки"]] = treeLayer2[layer2["Спортивные брюки"], layer2["Брюки"]] = 1
treeLayer2[layer2["Брюки"], layer2["Спортивные толстовки"]] = treeLayer2[layer2["Спортивные толстовки"], layer2["Брюки"]] = 0.8
treeLayer2[layer2["Брюки"], layer2["Свитеры и кофты"]] = treeLayer2[layer2["Свитеры и кофты"], layer2["Брюки"]] = 0.9
treeLayer2[layer2["Брюки"], layer2["Футболки"]] = treeLayer2[layer2["Футболки"], layer2["Брюки"]] = 0.7
treeLayer2[layer2["Брюки"], layer2["Лонгсливы"]] = treeLayer2[layer2["Лонгсливы"], layer2["Брюки"]] = 0.7
treeLayer2[layer2["Брюки"], layer2["Джинсы"]] = treeLayer2[layer2["Джинсы"], layer2["Брюки"]] = 0.6
treeLayer2[layer2["Брюки"], layer2["Шорты"]] = treeLayer2[layer2["Шорты"], layer2["Брюки"]] = 1

treeLayer2[layer2["Рубашки"], layer2["Спортивные футболки"]] = treeLayer2[layer2["Спортивные футболки"], layer2["Рубашки"]] = 1
treeLayer2[layer2["Рубашки"], layer2["Спортивные шорты"]] = treeLayer2[layer2["Спортивные шорты"], layer2["Рубашки"]] = 1
treeLayer2[layer2["Рубашки"], layer2["Спортивные брюки"]] = treeLayer2[layer2["Спортивные брюки"], layer2["Рубашки"]] = 1
treeLayer2[layer2["Рубашки"], layer2["Спортивные толстовки"]] = treeLayer2[layer2["Спортивные толстовки"], layer2["Рубашки"]] = 1
treeLayer2[layer2["Рубашки"], layer2["Свитеры и кофты"]] = treeLayer2[layer2["Свитеры и кофты"], layer2["Рубашки"]] = 0.8
treeLayer2[layer2["Рубашки"], layer2["Футболки"]] = treeLayer2[layer2["Футболки"], layer2["Рубашки"]] = 1
treeLayer2[layer2["Рубашки"], layer2["Лонгсливы"]] = treeLayer2[layer2["Лонгсливы"], layer2["Рубашки"]] = 1
treeLayer2[layer2["Рубашки"], layer2["Джинсы"]] = treeLayer2[layer2["Джинсы"], layer2["Рубашки"]] = 0.6
treeLayer2[layer2["Рубашки"], layer2["Шорты"]] = treeLayer2[layer2["Шорты"], layer2["Рубашки"]] = 1

treeLayer2[layer2["Спортивные футболки"], layer2["Спортивные шорты"]] = treeLayer2[layer2["Спортивные шорты"], layer2["Спортивные футболки"]] = 0.3
treeLayer2[layer2["Спортивные футболки"], layer2["Спортивные брюки"]] = treeLayer2[layer2["Спортивные брюки"], layer2["Спортивные футболки"]] = 0.3
treeLayer2[layer2["Спортивные футболки"], layer2["Спортивные толстовки"]] = treeLayer2[layer2["Спортивные толстовки"], layer2["Спортивные футболки"]] = 0.2
treeLayer2[layer2["Спортивные футболки"], layer2["Свитеры и кофты"]] = treeLayer2[layer2["Свитеры и кофты"], layer2["Спортивные футболки"]] = 0.7
treeLayer2[layer2["Спортивные футболки"], layer2["Футболки"]] = treeLayer2[layer2["Футболки"], layer2["Спортивные футболки"]] = 0.4
treeLayer2[layer2["Спортивные футболки"], layer2["Лонгсливы"]] = treeLayer2[layer2["Лонгсливы"], layer2["Спортивные футболки"]] = 0.7
treeLayer2[layer2["Спортивные футболки"], layer2["Джинсы"]] = treeLayer2[layer2["Джинсы"], layer2["Спортивные футболки"]] = 0.6
treeLayer2[layer2["Спортивные футболки"], layer2["Шорты"]] = treeLayer2[layer2["Шорты"], layer2["Спортивные футболки"]] = 0.5

treeLayer2[layer2["Спортивные шорты"], layer2["Спортивные брюки"]] = treeLayer2[layer2["Спортивные брюки"], layer2["Спортивные шорты"]] = 0.4
treeLayer2[layer2["Спортивные шорты"], layer2["Спортивные толстовки"]] = treeLayer2[layer2["Спортивные толстовки"], layer2["Спортивные шорты"]] = 0.5
treeLayer2[layer2["Спортивные шорты"], layer2["Свитеры и кофты"]] = treeLayer2[layer2["Свитеры и кофты"], layer2["Спортивные шорты"]] = 0.8
treeLayer2[layer2["Спортивные шорты"], layer2["Футболки"]] = treeLayer2[layer2["Футболки"], layer2["Спортивные шорты"]] = 0.4
treeLayer2[layer2["Спортивные шорты"], layer2["Лонгсливы"]] = treeLayer2[layer2["Лонгсливы"], layer2["Спортивные шорты"]] = 0.7
treeLayer2[layer2["Спортивные шорты"], layer2["Джинсы"]] = treeLayer2[layer2["Джинсы"], layer2["Спортивные шорты"]] = 1
treeLayer2[layer2["Спортивные шорты"], layer2["Шорты"]] = treeLayer2[layer2["Шорты"], layer2["Спортивные шорты"]] = 0.4

treeLayer2[layer2["Спортивные брюки"], layer2["Спортивные толстовки"]] = treeLayer2[layer2["Спортивные толстовки"], layer2["Спортивные брюки"]] = 0.4
treeLayer2[layer2["Спортивные брюки"], layer2["Свитеры и кофты"]] = treeLayer2[layer2["Свитеры и кофты"], layer2["Спортивные брюки"]] = 0.7
treeLayer2[layer2["Спортивные брюки"], layer2["Футболки"]] = treeLayer2[layer2["Футболки"], layer2["Спортивные брюки"]] = 0.5
treeLayer2[layer2["Спортивные брюки"], layer2["Лонгсливы"]] = treeLayer2[layer2["Лонгсливы"], layer2["Спортивные брюки"]] = 0.9
treeLayer2[layer2["Спортивные брюки"], layer2["Джинсы"]] = treeLayer2[layer2["Джинсы"], layer2["Спортивные брюки"]] = 1
treeLayer2[layer2["Спортивные брюки"], layer2["Шорты"]] = treeLayer2[layer2["Шорты"], layer2["Спортивные брюки"]] = 0.9

treeLayer2[layer2["Спортивные толстовки"], layer2["Свитеры и кофты"]] = treeLayer2[layer2["Свитеры и кофты"], layer2["Спортивные толстовки"]] = 0.6
treeLayer2[layer2["Спортивные толстовки"], layer2["Футболки"]] = treeLayer2[layer2["Футболки"], layer2["Спортивные толстовки"]] = 0.4
treeLayer2[layer2["Спортивные толстовки"], layer2["Лонгсливы"]] = treeLayer2[layer2["Лонгсливы"], layer2["Спортивные толстовки"]] = 0.6
treeLayer2[layer2["Спортивные толстовки"], layer2["Джинсы"]] = treeLayer2[layer2["Джинсы"], layer2["Спортивные толстовки"]] = 0.4
treeLayer2[layer2["Спортивные толстовки"], layer2["Шорты"]] = treeLayer2[layer2["Шорты"], layer2["Спортивные толстовки"]] = 0.8

treeLayer2[layer2["Свитеры и кофты"], layer2["Футболки"]] = treeLayer2[layer2["Футболки"], layer2["Свитеры и кофты"]] = 0.7
treeLayer2[layer2["Свитеры и кофты"], layer2["Лонгсливы"]] = treeLayer2[layer2["Лонгсливы"], layer2["Свитеры и кофты"]] = 0.6
treeLayer2[layer2["Свитеры и кофты"], layer2["Джинсы"]] = treeLayer2[layer2["Джинсы"], layer2["Свитеры и кофты"]] = 0.5
treeLayer2[layer2["Свитеры и кофты"], layer2["Шорты"]] = treeLayer2[layer2["Шорты"], layer2["Свитеры и кофты"]] = 1

treeLayer2[layer2["Футболки"], layer2["Лонгсливы"]] = treeLayer2[layer2["Лонгсливы"], layer2["Футболки"]] = 0.5
treeLayer2[layer2["Футболки"], layer2["Джинсы"]] = treeLayer2[layer2["Джинсы"], layer2["Футболки"]] = 0.5
treeLayer2[layer2["Футболки"], layer2["Шорты"]] = treeLayer2[layer2["Шорты"], layer2["Футболки"]] = 0.5

treeLayer2[layer2["Лонгсливы"], layer2["Джинсы"]] = treeLayer2[layer2["Джинсы"], layer2["Лонгсливы"]] = 0.5
treeLayer2[layer2["Лонгсливы"], layer2["Шорты"]] = treeLayer2[layer2["Шорты"], layer2["Лонгсливы"]] = 1

treeLayer2[layer2["Джинсы"], layer2["Шорты"]] = treeLayer2[layer2["Шорты"], layer2["Джинсы"]] = 1


#### Матрица смежности для категорий (3 уровень)

In [91]:
layer3 = {
    "Рубашки с коротким рукавом": 0,
    "Рубашки с длинным рукавом": 1,
    "Спортивные классические брюки": 2,
    "Джоггеры": 3,
    "Тайцы": 4,
    "Свитшоты": 5,
    "Худи": 6,
    "Толстовки": 7,
}

treeLayer3 = np.zeros((len(layer3), len(layer3)))

treeLayer3[layer3["Рубашки с коротким рукавом"], layer3["Рубашки с длинным рукавом"]] = treeLayer3[layer3["Рубашки с длинным рукавом"], layer3["Рубашки с коротким рукавом"]] = 0.2
treeLayer3[layer3["Рубашки с коротким рукавом"], layer3["Спортивные классические брюки"]] = treeLayer3[layer3["Спортивные классические брюки"], layer3["Рубашки с коротким рукавом"]] = 1
treeLayer3[layer3["Рубашки с коротким рукавом"], layer3["Джоггеры"]] = treeLayer3[layer3["Джоггеры"], layer3["Рубашки с коротким рукавом"]] = 0.9
treeLayer3[layer3["Рубашки с коротким рукавом"], layer3["Тайцы"]] = treeLayer3[layer3["Тайцы"], layer3["Рубашки с коротким рукавом"]] = 1
treeLayer3[layer3["Рубашки с коротким рукавом"], layer3["Свитшоты"]] = treeLayer3[layer3["Свитшоты"], layer3["Рубашки с коротким рукавом"]] = 1
treeLayer3[layer3["Рубашки с коротким рукавом"], layer3["Худи"]] = treeLayer3[layer3["Худи"], layer3["Рубашки с коротким рукавом"]] = 1
treeLayer3[layer3["Рубашки с коротким рукавом"], layer3["Толстовки"]] = treeLayer3[layer3["Толстовки"], layer3["Рубашки с коротким рукавом"]] = 0.9

treeLayer3[layer3["Рубашки с длинным рукавом"], layer3["Спортивные классические брюки"]] = treeLayer3[layer3["Спортивные классические брюки"], layer3["Рубашки с длинным рукавом"]] = 1
treeLayer3[layer3["Рубашки с длинным рукавом"], layer3["Джоггеры"]] = treeLayer3[layer3["Джоггеры"], layer3["Рубашки с длинным рукавом"]] = 1
treeLayer3[layer3["Рубашки с длинным рукавом"], layer3["Тайцы"]] = treeLayer3[layer3["Тайцы"], layer3["Рубашки с длинным рукавом"]] = 1
treeLayer3[layer3["Рубашки с длинным рукавом"], layer3["Свитшоты"]] = treeLayer3[layer3["Свитшоты"], layer3["Рубашки с длинным рукавом"]] = 0.7
treeLayer3[layer3["Рубашки с длинным рукавом"], layer3["Худи"]] = treeLayer3[layer3["Худи"], layer3["Рубашки с длинным рукавом"]] = 1
treeLayer3[layer3["Рубашки с длинным рукавом"], layer3["Толстовки"]] = treeLayer3[layer3["Толстовки"], layer3["Рубашки с длинным рукавом"]] = 1

treeLayer3[layer3["Спортивные классические брюки"], layer3["Джоггеры"]] = treeLayer3[layer3["Джоггеры"], layer3["Спортивные классические брюки"]] = 0.6
treeLayer3[layer3["Спортивные классические брюки"], layer3["Тайцы"]] = treeLayer3[layer3["Тайцы"], layer3["Спортивные классические брюки"]] = 0.8
treeLayer3[layer3["Спортивные классические брюки"], layer3["Свитшоты"]] = treeLayer3[layer3["Свитшоты"], layer3["Спортивные классические брюки"]] = 0.8
treeLayer3[layer3["Спортивные классические брюки"], layer3["Худи"]] = treeLayer3[layer3["Худи"], layer3["Спортивные классические брюки"]] = 0.4
treeLayer3[layer3["Спортивные классические брюки"], layer3["Толстовки"]] = treeLayer3[layer3["Толстовки"], layer3["Спортивные классические брюки"]] = 0.4

treeLayer3[layer3["Джоггеры"], layer3["Тайцы"]] = treeLayer3[layer3["Тайцы"], layer3["Джоггеры"]] = 0.9
treeLayer3[layer3["Джоггеры"], layer3["Свитшоты"]] = treeLayer3[layer3["Свитшоты"], layer3["Джоггеры"]] = 0.6
treeLayer3[layer3["Джоггеры"], layer3["Худи"]] = treeLayer3[layer3["Худи"], layer3["Джоггеры"]] = 0.6
treeLayer3[layer3["Джоггеры"], layer3["Толстовки"]] = treeLayer3[layer3["Толстовки"], layer3["Джоггеры"]] = 0.5

treeLayer3[layer3["Тайцы"], layer3["Свитшоты"]] = treeLayer3[layer3["Свитшоты"], layer3["Тайцы"]] = 1
treeLayer3[layer3["Тайцы"], layer3["Худи"]] = treeLayer3[layer3["Худи"], layer3["Тайцы"]] = 0.8
treeLayer3[layer3["Тайцы"], layer3["Толстовки"]] = treeLayer3[layer3["Толстовки"], layer3["Тайцы"]] = 0.7

treeLayer3[layer3["Свитшоты"], layer3["Худи"]] = treeLayer3[layer3["Худи"], layer3["Свитшоты"]] = 0.3
treeLayer3[layer3["Свитшоты"], layer3["Толстовки"]] = treeLayer3[layer3["Толстовки"], layer3["Свитшоты"]] = 0.5

treeLayer3[layer3["Худи"], layer3["Толстовки"]] = treeLayer3[layer3["Толстовки"], layer3["Худи"]] = 0.3

In [92]:
layer = [layer1, layer2, layer3]
tree = [treeLayer1, treeLayer2, treeLayer3]

#### Данные

In [93]:
excludeFields = ["Цвет", "Бренд"]

def getDataFrameStat(df):
    dfStat = df.copy()
    for elem in excludeFields:
        del dfStat[elem]
    
    return dfStat

#### Получение расстояний

In [94]:
def _getDistance(v1, v2, nPow):
    res = 0
    for i in range(len(v1)):
        if isnan(v1[i]) or isnan(v2[i]):
            continue
        res += pow(abs(v1[i] - v2[i]), nPow)
        
    return pow(res, 1 / nPow)

# Манхэттенское расстояние
def getManhattanDistance(v1, v2):
    return _getDistance(v1, v2, 1)

# Евклидово расстояние
def getEuclideanDistance(v1, v2):
    return _getDistance(v1, v2, 2)

# Косинусное 
def getCos(v1, v2):
    v1T, v2T = v1.copy(), v2.copy()
    indArr = [i for i, (elem1, elem2) in enumerate(zip(v1T, v2T)) if isnan(elem1) or isnan(elem2)]
    v1T[:] = [elem for i, elem in enumerate(v1T) if i not in indArr]
    v2T[:] = [elem for i, elem in enumerate(v2T) if i not in indArr]

    return 1 - dot(v1T, v2T) / (norm(v1T) * norm(v2T))

# Расстояние по дереву
def getTreeDistance(v1, v2):
    res = 0
    size = max(len(v1), len(v2))

    for i in range(size):
        try:
            res += tree[i][layer[i][v1[i]]][layer[i][v2[i]]]
        except:
            res += 0.5
    
    return res / size

# Сравнение брендов
def getBrandDistance(v1, v2):
    return brandsMatr[brands[v1]][brands[v2]]

# Сравнение цветов
def getColorDistance(v1, v2):
    return colorsMatr[colors[v1]][colors[v2]]

# Найти все похожие
def getSimilarity(id, matr, nameArr):
    data = matr[id]
    res = pd.DataFrame(
        zip(data, nameArr), 
        index=np.arange(len(matr)), 
        columns=["Расстояние", "Название"]
    )
    return res.sort_values("Расстояние")


In [95]:
def calcDistance(f, df):
    matrData = df.values.tolist()
    n = len(matrData)
    matrRes = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            matrRes[i][j] = matrRes[j][i] = f(matrData[i], matrData[j])
            
    return matrRes / matrRes.max()   


def calcDistanceCompined(df, dfTree):
    dfTree = dfTree["Иерархия"]
    dfStatParams = getDataFrameStat(df)
    
    matrTree = calcDistance(getTreeDistance, dfTree)
    matrEucl = calcDistance(getEuclideanDistance, dfStatParams)
    # matrMan = calcDistance(getManhattanDistance, dfStatParams)
    # matrCos = calcDistance(getCos, dfStatParams)
    matrBrand = calcDistance(getBrandDistance, df["Бренд"])
    matrColor = calcDistance(getColorDistance, df["Цвет"])

    xTree = matrTree.max()
    xStat = matrEucl.max()
    # xStat = matrMan.max()
    # xStat = matrCos.max()
    xBrand = matrBrand.max()
    xColor = matrColor.max()

    kTree, kStat, kBrand, kColor = 4, 4, 1, 2

    return (kTree * matrTree + kStat * matrEucl + kBrand * matrBrand + kColor * matrColor) / \
        (kTree * xTree + kStat * xStat + kBrand * xBrand + kColor * xColor)


#### Графическое изображение

In [96]:
def draw(matrRes, nameArr, title, color='Inferno'):
    fig = px.imshow(matrRes, x=nameArr, y=nameArr, color_continuous_scale=color, title=title)
    fig.update_layout(width=1000, height=1200)
    fig.update_traces(text=nameArr)
    fig.update_xaxes(side="top")
    fig.show()

### Демонстрация расстояний

#### Манхэттенское расстояние


In [97]:
matrRes = calcDistance(getManhattanDistance, getDataFrameStat(df))
draw(matrRes, nameArr, "Манхэттенское расстояние")

#### Евклидово расстояние

In [98]:
matrRes = calcDistance(getEuclideanDistance, getDataFrameStat(df))
draw(matrRes, nameArr, "Евклидово расстояние")

#### Косинусная мера близости

In [99]:
matrRes = calcDistance(getCos, getDataFrameStat(df))
draw(matrRes, nameArr, "Косинусная мера близости")

#### Расстояние по брендам

In [100]:
matrRes = calcDistance(getBrandDistance, df["Бренд"])
draw(matrRes, nameArr, "Расстояние по брендам")

#### Расстояние по цветам

In [101]:
matrRes = calcDistance(getColorDistance, df["Цвет"])
draw(matrRes, nameArr, "Расстояние по цветам")

#### Расстояние по дереву

In [102]:
matrRes = calcDistance(getTreeDistance, dfTree["Иерархия"])
draw(matrRes, nameArr, "Расстояние по дереву")

#### Комбинированная мера

In [103]:
draw(calcDistanceCompined(df, dfTree), nameArr, "Комбинированная мера")

#### Требуемые параметры

In [104]:
matrSimilarity = calcDistanceCompined(df, dfTree)

### Задача 1

**Вход**: 1 объект (затравочный). 

**Выход**: список рекомендаций, ранжированный по убыванию близости с затравкой. Примените Вашу обобщающую меру близости.

In [105]:
def printRes(arr):
    print("Расстояние \t\t\t Название")
    for elem in arr:
        for key, value in elem.items():
            print("{0}\t\t{1}".format(value, key))

In [106]:
def _findSimilar(name):
    ind = df0[F_NAME].tolist().index(name)
    
    listSimilarity = getSimilarity(ind, matrSimilarity, nameArr)
    return listSimilarity


def findSimilar(name):
    listSimilarity = _findSimilar(name)

    return listSimilarity[listSimilarity[F_NAME] != name]

res = findSimilar("Спортивная футболка Puma Синий 3200")
print(res)

    Расстояние                                 Название
31    0.077622  Спортивная футболка Reebok Голубой 3800
38    0.082518         Спортивные шорты Puma Синий 4400
70    0.082984     Спортивная толстовка Puma Синий 5200
35    0.093939       Спортивные шорты Reebok Синий 4500
30    0.118415     Спортивная футболка Puma Черный 3600
..         ...                                      ...
3     0.772960                 Брюки Canali Серый 11400
4     0.792541                Брюки Briony Черный 13800
5     0.834033             Брюки Corneliani Белый 14800
2     0.838228              Брюки Hugo Boss Серый 14200
6     0.869930            Брюки Hugo Boss Розовый 14000

[127 rows x 2 columns]


### Задача 2

**Вход**: массив объектов (лайков). 

**Выход**: сформированный ранжированный список рекомендаций.

In [107]:
def _findSimilarMany(nameArr):
    recList = []
    for name in nameArr:
        rec = _findSimilar(name)
        recList.append(rec.loc[rec[F_NAME].isin(nameArr) == False])

    dfRes = defaultdict(lambda: 1e2)
    for rec in recList:
        for i, row in rec.iterrows():
            curName = row[F_NAME]
            curDist = row[F_DIST]
            dfRes[curName] = min(dfRes[curName], curDist)

    return dfRes


def findSimilarMany(nameArr):
    resDict = _findSimilarMany(nameArr)
    return sorted(
        [{key: elem} for key, elem in resDict.items()], 
        key=lambda elem: list(elem.values())[0]
    )

res = findSimilarMany([
    "Спортивная футболка Nike Черный 3100",
    "Спортивная толстовка Adidas Серый 6000"
])
printRes(res)

Расстояние 			 Название
0.020747434912845673		Спортивная футболка Puma Черный 3600
0.04871805539960568		Спортивная толстовка Nike Серый 4300
0.04871806225547481		Спортивная толстовка Puma Серый 4300
0.06853154516812089		Спортивная толстовка Adidas Черный 5400
0.07062952549618579		Спортивные шорты Puma Черный 3400
0.08228445371045251		Спортивная толстовка Nike Черный 5200
0.0916083991944917		Спортивные шорты Reebok Черный 4300
0.0939397233535078		Спортивные шорты Adidas Черный 4400
0.09627039378958785		Спортивная футболка Adidas Серый 1700
0.11328671204630908		Тайцы Nike Черный 3800
0.1205146843920184		Спортивная футболка Puma Синий 3200
0.12517734943977288		Спортивная футболка Reebok Красный 3400
0.12750690482320146		Джоггеры Reebok Серый 5600
0.14149185137484335		Спортивные брюки Puma Серый 5000
0.1414918979948165		Джоггеры Nike Серый 5000
0.14382288425545953		Тайцы Puma Серый 4900
0.14382288425545953		Спортивные брюки Puma Серый 7100
0.14382289485090416		Спортивная футболка Adidas Кр

### Задача 3

**Вход**: массив затравочных объектов и массив дизлайков.

**Выход**: сформированный ранжированный список рекомендаций.

In [108]:
def delOpposite(dict, nameArr):
    for name in nameArr:
        if name in dict.keys():
            del dict[name]
    
    return dict


def findByReaction(likesArr=[], dislikesArr=[]):
    likesRec = delOpposite(_findSimilarMany(likesArr), dislikesArr)
    dislikesRec = delOpposite(_findSimilarMany(dislikesArr), likesArr)

    dictRes = {}
    if len(likesArr) == 0:
        for key, elem in dislikesRec.items():
            dictRes[key] = 1 - elem
        return sorted(
            [{key: elem} for key, elem in dictRes.items()], 
            key=lambda elem: list(elem.values())[0]
        )

    for key in likesRec.keys():
        if likesRec[key] <= dislikesRec[key]:
            dictRes[key] = likesRec[key]

    return sorted(
        [{key: elem} for key, elem in dictRes.items()], 
        key=lambda elem: list(elem.values())[0]
    )


res = findByReaction(
    likesArr=["Брюки Hugo Boss Серый 14200"],
    dislikesArr=["Рубашка с коротким рукавом Canali Синий 2900"]
)
printRes(res)

Расстояние 			 Название
0.072960517938067		Брюки Briony Черный 13800
0.07435899020987345		Брюки Canali Серый 11400
0.11398632371964429		Брюки Corneliani Белый 14800
0.16153908257980987		Брюки Corneliani Синий 14500
0.1745920834904174		Брюки Canali Белый 11000
0.1864811766964637		Брюки Hugo Boss Розовый 14000
0.2118883304423548		Брюки Briony Голубой 13300
0.29370630506149625		Джинсы Levi's Серый 15100
0.3286714021185321		Джинсы H&M Серый 16600
0.42424242168977777		Джинсы Levi's Серый 7700
0.4811189454011575		Джинсы H&M Красный 11500
0.5067599521671062		Джинсы Zara Красный 10400
0.5477855449198615		Спортивные брюки Adidas Серый 7600
0.5594405731241278		Спортивные брюки Puma Серый 7100
0.5944056022053161		Джоггеры Reebok Серый 5600
0.6083915933560021		Джоггеры Nike Серый 5000
0.608391598423398		Спортивные брюки Puma Серый 5000
0.6093240004789311		Спортивная толстовка Adidas Серый 6000
0.6107226142945854		Тайцы Puma Серый 4900
